In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
#from plotly.offline import init_notebook_mode, iplot
#import plotly.graph_objs as go
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [3]:
##### 파일 불러오기 #####
df = pd.read_csv("./data_preprocessing/4_card_spending_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'card_cnt', 'card_amt']  ##### 각자 데이터에 맞게 열이름 변경하기 #####
df

,base_date,emd_cd,card_cnt,card_amt
base_date,,,,
2018-01-31,2018-01-31,50110250,215.794788,8171132.984
2018-02-28,2018-02-28,50110250,213.713262,8605532.333
2018-03-31,2018-03-31,50110250,228.401338,8517373.716
2018-04-30,2018-04-30,50110250,239.895973,8640048.185
2018-05-31,2018-05-31,50110250,250.348387,9384709.810
...,...,...,...,...
2021-02-28,2021-02-28,50130620,65.216000,2463788.712
2021-03-31,2021-03-31,50130620,67.700000,2572585.700
2021-04-30,2021-04-30,50130620,68.642336,2473174.901


In [4]:
# 필요 없는 열 제거
data = df.drop(['base_date', 'card_amt'], 1)     ##### 각자 필요없는 열 제거 #####
data

,emd_cd,card_cnt
base_date,,
2018-01-31,50110250,215.794788
2018-02-28,50110250,213.713262
2018-03-31,50110250,228.401338
2018-04-30,50110250,239.895973
2018-05-31,50110250,250.348387
...,...,...
2021-02-28,50130620,65.216000
2021-03-31,50130620,67.700000
2021-04-30,50130620,68.642336


In [354]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130620

# base_date, card_cnt 만 갖는 df 생성

sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df.head()

,card_cnt
base_date,
2018-01-31,62.371212
2018-02-28,58.235537
2018-03-31,58.512367
2018-04-30,61.879699
2018-05-31,64.714286


In [355]:
# series 형으로 변환
sub_df.to_dict('series')
sub_df.head()

,card_cnt
base_date,
2018-01-31,62.371212
2018-02-28,58.235537
2018-03-31,58.512367
2018-04-30,61.879699
2018-05-31,64.714286


# ARIMA

In [356]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)
#sub_df_float.tail()

In [357]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
# from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 p-value 확인 -> 0.05보다 크면 아래로 내려가서 차분하기 / 작으면 차분 생략(아래 셀 생략) #####

ADF Statistic : -1.327659
p-value : 0.616425
Critical Values :
	1%: -3.639224
	5%: -2.951230
	10%: -2.614447


In [358]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 또 p-value 확인 -> 0.05보다 크면 이 파일 안되고 log파일로 넘어가세요 / 작으면 아래로 쭉 진행 #####

ADF Statistic : -6.132382
p-value : 0.000000
Critical Values :
	1%: -3.639224
	5%: -2.951230
	10%: -2.614447


In [359]:
# from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

##### 여기서 아래쪽에 'Best model:  ARIMA(0,1,0)(0,0,0)[1] intercept' 이렇게 나오면 (0,1,0) 가져다 쓸거임 #####
##### ARIMA(0,1,0) 여기 숫자 세 개만 잘 보면 됨 !!!! #####

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=254.306, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=242.276, Time=0.05 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=232.547, Time=0.05 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.31 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.25 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.23 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=231.706, Time=0.04 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=233.558, Time=0.13 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=233.833, Time=0.20 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.17 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.23 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=233.425, Time=0.07 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=232.765, Time=0.09 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=233.689, Time=0.18 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.21 sec
 ARIMA(3,0,0)(0,0,0)[1] intercept   : AIC=231.804

ARIMA(order=(5, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [360]:
####################################
######### order 바꾸기 ############
###################################
# ARIMA 모델 
# from statsmodels.tsa.arima_model import ARIMA
# import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.card_cnt.values, order=(5,0,0))    ##### 여기 order=(0,1,0)에 위에서 나온 숫자 넣는거 #####
                                    ##### 그리고 sub_df.card_amt.values에서 가운데 card_amt 각자걸로 바꾸기 #####

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\pione\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(5, 0)   Log Likelihood                -106.046
Method:                       css-mle   S.D. of innovations              2.946
Date:                Sat, 11 Sep 2021   AIC                            226.092
Time:                        02:23:14   BIC                            238.256
Sample:                             0   HQIC                           230.551
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         68.1190      0.697     97.776      0.000      66.754      69.484
ar.L1.y        0.5618      0.133      4.225      0.000       0.301       0.822
ar.L2.y        0.1727      0.157      1.100      0.2

# 예측하기

In [361]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

##### 맨 앞에 나온 'array([10681271.81014634, 10741037.02029268])' 이게 차례대로 7, 8월 예측값임 #####
##### 7월 : 10681271.81014634, 8월 : 10741037.02029268 #####
##### 이 값 가져다 엑셀파일에 입력하기 #####

(array([70.50166442, 69.88811482]), array([2.94601925, 3.37908676]), array([[64.72757279, 76.27575606],
       [63.26522646, 76.51100318]]))
